# LLM基础分享
## 一些基础的理论知识
  
  LLM(Large Language  Model)，即大型语言模型，本质上是一个深度神经网络，因此我们先来聊聊神经网络的一些基础知识，以便你对后面的一些具体细节有更为理性一些的认知。在计算机科学中，我们所谓的神经网络，指代的一般就是人工神经网络(ANN:Artificial Neural Network)。人工神经网络来自于对生物神经网络的模仿，是一种数学模型或者计算模型，用于对函数进行估计和近似。正如同生物神经网络一样，人工神经网络也是由一个个人工神经元组成的，每个人工神经元，有如下的简单结构:

<center>
    <img src="https://llmshare.zhoudailin.cn/pictures/Ncell.png?imageMogr2/thumbnail/400x"/>
</center>

1. 其中$a_1$到$a_n$为输入的各个分量
2. $w_1$到$w_n$是各个分量的权重
3. b为偏置值
4. f称为激活函数，一般是非线性函数，常用激活函数有sigmoid，tanh,Relu等

如果将a和w作为一个整体(向量)，其实神经元就代表了一个很简单的函数
$$t=f(wa+b)$$
那么将很多神经元连接起来，就可以得到各式各样的神经网络

<center>
    <img src="https://llmshare.zhoudailin.cn/pictures/ANN.png?imageMogr2/thumbnail/400x"/>
</center>

那么这些各种连接的神经元为啥能够产生所谓的智能呢，这时候就需要提到如下的一个定理
> 通用近似定理：固定一元函数和一组仿射泛函的有限线性组合可以一致逼近单位超立方体中的任意连续函数

其中的一元函数说的就是上面公式的$wa+b$部分，而仿射泛函就是指的激活函数，有限线性组合指的就是有限个神经元连接在一起。通用近似定理其实翻译成人话就是理论上证明了**不管一个连续函数多么复杂，都可以通过神经网络模拟出来**。而所谓智能其实就可以看成输入和输出的一个隐含关系。比如我们在使用LLM的时候，LLM本质就是一个将你的文字输入映射到另外一段文字输入的函数。而比如人脸识别，就是一个图像到一个身份信息，比如id的一个映射关系。

通用近似定理告诉了我们神经网络可以模拟任何函数，却没有告诉我们如何得到这个函数。其实从上面描述我们可以看出来，如果我们确定了神经网络的连接方式，激活函数，权重$w$和偏置$b$等，那么就确定了一个神经网络了。一般来说，连接方式和激活函数是一个神经网络架构上的选择，不同名字的神经网络，比如GAN，RNN这些一般都有不同的连接方式和激活函数选择。而一旦确定了神经网络的架构，我们剩余的就是要**确定权重$w$和偏置$b$，而确定他们的过程，就是所谓的训练**。

我们在后面选择模型的时候，会看到同一个模型会有不同的标识，比如`7B,20B,120B`等等，这个标识其实说的就是参数规模，这个参数主要指的就是权重和偏置。其中B指的是Bilion，所以比如一个神经网络写了7B，就表示它的参数规模是70亿左右。

有了网络结构，有了参数规模，那么就是如何得到权重和偏置了。这里我简单解释下训练的过程是如何进行的，这个对于后面理解什么是微调是有帮助的。一般来说我们把机器学习可以通过是否给定答案来分为`有监督学习`和`无监督学习`。所谓有监督学习，就是你要告诉神经网络，你的输入正确的输出是什么，而无监督学习就只需要告诉输入。一般来说大部分任务都是需要监督学习完成的，因此我们平时会看到各种打标记，这个过程其实就是给神经网络准备正确答案的过程。

当我们准备好了训练的数据，也就是一堆你所希望神经网络展现的正确的输入和输出之后，会将一定比例的数据留下用于验证最后结果，其他数据就会依次给神经网络用于训练。一般来说训练就是如下的一个过程
1. 虽然初始化上面的各种参权重，偏置等参数
2. 将某条数据的输入给到神经网络，得到这时候的输出
3. 这时候的输出肯定和正确的输出是不一样的(否则就不需要训练了),那么可以通过某个指标计算出此时神经网络给出的结果和正确结果的误差有多大。
4. 根据误差情况调整各种参数的值
5. 重复上面2到4的步骤，直到数据都训练完成，或者输出的误差小于预定的一个目标
6. 将剩余的验证用的数据输入此时的神经网络评估结果

其实可以看出来，所谓的训练，就是通过调整参数的值，去尽量让神经网络给出的结果误差和正确误差越来越小的过程

## LLM目前情况综述
在知道了一些必要的基础知识之后，我们回到大语言模型本身。大语言模型(非多模态)属于人工智能自然语言处理(NLP)这个领域，其实这个领域历史很悠久，它的很多成果我们也一直在不知不觉中使用着。比如我们用过ElasticSearch，其中有个部分就是需要分词器，而分词就是NLP一个很重要的话题。

其实在谷歌2017年发表《Attention Is All You Need》这篇论文之前，自然语言处理是没有计算机视觉火热的。当时GAN的发明，应该有人有印象当时有很多做风格迁移的，就是你给两张图片，比如一张是梵高的星空，一张是普通的照片，就会有个程序帮你把那种普通的照片变成梵高星空的风格。后来谷歌提出注意力机制之后不久又提出了Bert，这个当时也火了一段时间之后就没啥下文了，直到2023年ChatGPT的出现，让自然语言处理火出了圈。

如今仅仅过去了一年左右，世面上优秀的大语言模型早已不是ChatGPT一家独大，而是呈现了一种百花齐放的姿态，其中最出名的有如下这些模型


| 模型名 | 是否开源|是否多模态 | 开发商|许可证 |特点|
|:--------:| :---------:|:---:|:--------:|:----:|:----:|
| ChatGPT系列 | 否 |GPT4及其后面模型支持|OpenAI|/|遥遥领先|
|Claude|否|claude3|Anthropic|/|前OpenAI高管创立，支持多模态，上下文窗口极大|
|Llama系列|是|是|Meta|Llama 3 license|开源之王，从Llama衍生出了大量的二创作品，但是中文训练数据不太多，经常用英文回答|
|Gemini系列|否|是|Google DeepMind|/|和前面的还是有差距，谷歌还是做基础研究写论文厉害|
|Gemma|是|是|Google DeepMind|Apache 2.0|同上|
|Mixtral 8x7B|是|是|MistralAI|Apache 2.0|和ChatGPT据说使用了同样的架构但是开源，最近很火|
|Phi-2|是|否|微软|MIT|小模型，参数量小，有望嵌入边缘设备。中文支持不好。|
|文心一言|否|是|百度|/|大量中文语料训练，实际表现一般|
|百川|是|否|百川智能| Apache 2.0 |对话,创作,生成能力非常不错，代码和逻辑能力不好，算是国内模型比较优秀的|
|Qwen|是|是|阿里云|Apache 2.0|系列模型，昨年第一个版本效果很差，最近2.5的质量不错|
|ChatGLM|是|是|清华大学|Apache 2.0|对中文支持各类评测中最优秀，模型规模小，很容易跑起来|

以上这些模型是从昨年到今年个人使用过的，其实还有很多著名的模型，比如腾讯的，商汤的，360的等，但是个人没深入使用过。上面的特点也只是个人感受，而且大模型这个东西，尤其是闭源的以服务形式提供的大模型，可能每天实际表现都会有偏差，所以以上也只能代表他们目前在个人心目中的情况。其实作为一个开发者而不是模型的研究者，个人认为不要将模型本身看得太过重要。当然这并不是说随便选个模型即可，毕竟LLM作为目前AI应用的心脏，重要性不言而喻。但是问题在于现在大模型层出不穷，将来可能会越来越多，如果作为开发者将所有时间都投入到了模型本身的研究上，有点本末倒置。何况目前因为langchain这类AI开发框架的成熟，在一个应用中将模型的选择和具体应用的实施解耦，使得替换一个模型变成一个很简单的事情。

## 模型的部署
一般来说，LLM都是python开发的应用程序，所以生产部署的话，实际就是走python部署的流程即可。注意这里的部署和后面的本地部署是有区别的，这里部署是以生产为目的，最终目标是跑起来暴露一套api给多人提供服务。而后面的本地部署，基本上意味着这个api只提供给单人服务。所以在使用的方案，工具，硬件需求等有很大的差别。

不同的模型其实在他的仓库里面基本都说明了如何去部署，这里主要提两个点：

第一个就是，前面说到了，预训练模型训练完成后会得到有参数，这个参数量是以十亿为单位的，很大。所以就有一个问题，这个参数存放在哪的，如果你去github看，会发现仓库里面不会有参数提供的，因为参数动辄几十G，大一点的上百G，甚至T级别。这里我们就要说到一个平台了，叫做huggingface。简单解释这是一个类似github的地方，被称为深度学习的github。但是和github不一样，这上面存放的不是代码，而是模型的参数，更准确地说，是transformer模型的参数。huggingface呢，写了个一个叫transformer的库，只要模型是transformer，就可以通过这个库加上参数运行。其实我们之前提过ChatGPT的T就是指代transformer，是谷歌提出来的基于注意力机制的一种模型架构。因为目前大部分各种各样的LLM基本都是transformer架构，所以他们都可以通过huggingface提供的库启动，也因此他们的参数都存放在了huggingface。我们以通义千问7B版本为例，首先是他的[github仓库](https://github.com/QwenLM/Qwen/tree/main)：

<center>
    <img src="https://llmshare.zhoudailin.cn/pictures/Qwen_github.png?imageMogr2/thumbnail/800x"/>
</center>

可以看出来这上面只有模型本身的一些代码，包括启动的，训练的，微调的，量化的等等。我们再去[huggingface看看](https://huggingface.co/Qwen/CodeQwen1.5-7B/tree/main)：

<center>
    <img src="https://llmshare.zhoudailin.cn/pictures/Qwen_huggingface.png?imageMogr2/thumbnail/800x"/>
</center>

会看到这上面有几个很大的文件，这些就是模型的权重信息，剩下的是一些描述模型结构等其他信息的文件。这样的话只需要有huggingface的transformer库，加上模型的名字就可以运行模型本身了，因为权重信息已经存放在huggingface了。其实即使你不直接通过huggingface去调用模型，你去看Qwen里面的代码，它封装好的api也是通过huggingface去调用的。通过如下代码就可以加载：

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-7B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-7B")
prompt = "简单介绍下你自己."

messages = [{"role": "user", "content": prompt}]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

如果你的显卡是4090这种有24G左右显存，你可以尝试运行上面代码，否则会出现显存不足。即使你显存充足，依然可能会遇到一个问题，就是huggingface的访问问题，在国内huggingface是不能直接访问的，这里一般有两种处理方式，第一种依然使用huggingface，但是通过镜像源，或者手动下载权重文件到本地，又或者挂代理等方式处理。另外一种直接放弃huggingface，使用其他平台，比如阿里的modelscope魔搭社区，就可以看成阿里做的国内版huggingface，虽然没有huggingface那么多能力，但是不会存在访问问题。这里我们通过镜像站的方式来解决这个问题，但是因为我的显卡是不够运行Qwen-7B这个模型的非量化版本的，我们去云平台租用一台GPU服务器来完成演示，顺便分享些云平台使用的小tips。

<center>
    <video src="https://llmshare.zhoudailin.cn/medias/%E9%83%A8%E7%BD%B2.mp4" width="800" controls></video>
</center>


## 如何微调模型
前面我们说了训练，大家也应该知道训练的本质究竟是啥。一般来说市面上的LLM模型的训练素材都不是特化了，除了辅助代码编程类的LLM可能会使用大量的代码进行训练。因此这些LLM一般来说是通用的，可以用于任何问题，但是这也意味着他们不具有某些领域知识，也不知道训练完成后新的知识。比如某个太具体的细分专业的知识，我是学电气工程的嘛，所以很多同学在国家电网，当时GPT4才出来的时候我就开了会员，然后让同学给了些电网的题目给GPT4回答，结果当然是一塌糊涂。又比如你问哪怕最好的大语言模型如何使用大禹，它可能会给你一本正经胡说八道，因为它根本不知道大禹是公司的一个内部基于k8s的系统。又比如说前不久GPT4o刚出来嘛，如果你这时候去问很多模型关于GPT4o的信息，他们应该也是不知道的。那么这时候我们就可以对模型进行微调。
想象一下，你现在手上有个语言模型，你很满意，但是你希望它能有一些前端的领域知识，但是它好像就是不懂前端。可能这个模型训练的时候假设用了10亿条数据，其中可能和前端相关的数据聊聊无几。它训练完成了的到了目前的参数。如果你这时候再去准备比如1万条前端的数据，在目前的参数的基础上继续进行训练，那么就约等于这1万条数据本来就在这10亿训练数据中一样，而你也不需要花费10亿条数据的训练时间，这部分时间前面的训练已经付出了。那么这个过程就是微调，**本质就是你准备一些新的数据，然后继续训练，在之前参数的基础上更新出一个新的参数的过程**，从上面微调的过程可以看出有以下隐含信息：
1. 每个模型训练的时候准备的数据格式是不一样的，因此你准备的数据格式要和你的基础模型训练的格式一致，这个基本每个模型下面都是`fine-tune`的指南，里面都会提到数据格式的问题
2. 你的数据量不能太多也不能太少。这个很好理解，假设你说我很有钱，无所谓之前10亿条数据花费多少时间，我再准备100亿条领域知识喂给模型，这样出来的模型结果可能就会将之前已经学到的东西给遗忘掉(产生所谓的过拟合)。当然你的数据量也不能太小，比如你就期望准备10条数据，让LLM学会领域知识，这个确实想想也不太靠谱。
3. 微调是一个很耗费时间和钱的过程，对于开发工程师来说除非你准备在这上面深耕，否则让更专业的人或者手上有更多资源的人来做可能会更好。因为微调本来其实是一个需要很多知识才能做到的事情，模型一般是python写成，然后使用了pytorch这样的框架，所以你至少得懂得python和pytorch。然后微调的过程中还有很多参数可以控制，这些参数如何设置，需要你多少对网络结构有一些感性认知。最后就是，其实从最开始的通用近似定理我们可以看出来，我们所谓的智能是一种不可以解释的涌现智能，而目前也没有任何定量的理论指导如何会让一个模型的表现变得更好，很多东西都是定性的，形而上的，带一些玄学色彩的，大家应该听过那些搞模型训练的人，把训练的过程称为"炼丹"，其实也是这个原因。虽然目前有很多开源的项目降低了微调需要的学习成本，但是微调还有另外一个巨大的硬件成本，这个是开源解决不了的，除非你有资源，有好的显卡。否则尝试一次的成本非常高，而且得到的结果也可能只能用于学习而已。

虽然上面不建议以开发应用为目的的工程师太过死磕微调，但是微调作为一个改变LLM行为的重要手段，也是需要掌握的。上面也提到过，训练和微调都是非常吃硬件资源的过程，所以很多研究者提出了一些微调的方法，可以降低微调过程中的硬件要求。同时也有很多开源项目的出现，直接降低了微调所需要的知识。

#### LoRA
首先第一种方法就是跟着之前训练的过程继续进行，调整所有参数，这个被称为`Full fine-tune`。因为需要调整所有参数，这些参数都需要占用显存，因此需要的硬件资源极高，但是也会带来相对最好的结果表现。这个对于企业来说如果有大量的显卡资源，肯定是最优选择，但是对于个人学习来说，我们更推荐其他微调方法，比如LoRA。接下来的示例我们也会以LoRA作为例子去微调一个Llama3。

所谓LoRA(Low Rank Adapter)其实是基于研究者发现，对于一个大型的神经网络模型，调整它部分参数可以达到和调整全部参数一样的效果，也就是有一个极小的内在维度。LoRA作者受此启发，认为训练参数有个很小的内在秩，因此可以通过矩阵分解来用少量的数据替代大量的数据。

为了通俗地解释LoRA的原理，我们回到最初的网络结构图。我们图示的示例网络是有三层，其中第一层是有3个神经元，第二层是有4个。它们之间一共有12，也就是$3\times 4$条连线。这些连线本质上代表的是啥呢回顾一开始的神经元，其实这些连线就是代表了权重$w$，如果你没有想通这点，也无所谓，反正知道每条线上应该有个隐藏的数字，这个数字决定了当前输出的多少比例进入到下个神经元。接下来就需要回顾一下你大学的线性代数知识了，这$3\times 4$个数字，可以表示为一个$3\times 4$的矩阵。其实神经网络的参数很多都是以矩阵形式作为整体计算的，因为GPU很擅长处理这种矩阵的并行运算。图上的结构有三层，我们可以从中看出两个矩阵，而且两个矩阵大小都不是很大。但是实际的大型模型，里面可能存在数十个甚至上百个这样的矩阵，且每个矩阵都非常大。

为了解释LoRA，你需要再回忆一件事，就是矩阵的乘法。我们知道两个矩阵$A$和$B$，如果$A$是$m*n$的矩阵，而$B$是$n*s$的矩阵，那么$AB$会得到$m*s$的矩阵：
<center>
    <img src="https://llmshare.zhoudailin.cn/pictures/%E7%9F%A9%E9%98%B5%E4%B9%98%E6%B3%95.png" width="400"/>
</center>

有了这些准备，那么LoRA就很好理解了，其实它就是通过反过来使用矩阵乘法，将一个大矩阵分解成了两个小矩阵的乘积。假设我们需要训练的模型里面有个$10000*10000$的巨大矩阵，我们可以将它分解为一个$10000*1$和一个$1*10000$的矩阵，这样我们需要调整的参数就从$10000*10000$变成了$10000*2$。这个1其实就是所谓的秩，这也就是LoRA名字低秩适配器的由来。当然[实际情况](https://zhuanlan.zhihu.com/p/646791309)下，我们的秩可能取值不一定取1那么极端，但是根据实际结果表明，即使对很小的秩，比如2到9这种个位数，LoRA也有很好的效果。

明白了理论，我们就来实际操作下如何微调:

<center>
    <video src="https://llmshare.zhoudailin.cn/medias/%E5%BE%AE%E8%B0%83.mp4" width="800" controls></viedo>
</center>
        
## 如何量化模型
完成了微调之后，我们还有个问题没有处理，那就是如何低成本得让模型运行起来。这时候我们就需要对模型进行量化。所谓量化，简单来说就是压缩数据。从上面我们可以看到，最消耗资源的部分就是模型中巨量的参数，这些参数一般来说是16位浮点，或者32位浮点或者混合浮点(amp)，他们每个数字所占用的空间都比较大，如果我们能用比较小的空间来表示这些数字，就可以大大节约资源。当然，这种用短位数去表达长位数，肯定会带来舍入误差的，那么问题在于这个误差对模型效果有没有影响，影响有多大。第一个当然是有影响的，所以量化过后的模型一般都是用于本地学习或者本地日常使用，生产等要求比较高但是资源相对充分的环境还是尽量使用未量化的版本。第二点，影响有多大。因为量化其实是有个量化参数的，就像压缩率。比如我模型本来是16位浮点数。你用8位int去代替，这时候效果可能还可以，如果你换成了4位int，可能效果就会更差些。一般来说根据统计数据量化4Q(用4位int)的版本比非量化的效果会平均差10%-20%。这个结果对于本来来说肯定是可以容忍的。量化一般通过llama.cpp进行，我们来进行操作

<center>
    <video src="https://llmshare.zhoudailin.cn/medias/%E9%87%8F%E5%8C%96.mp4" width="800" controls></viedo>
</center>

## 如何本地部署和使用LLM
如果你只是一个使用者，或者只是想用api去完成你的某个作品，那么上面这些微调，量化可能对你来说不是太有吸引力，你需要的其实是一个可以调用的api。最初大家基本都使用ChatGPT的api，因为那时候ollama这种东西还没出现。虽然ChatGPT的账户每个都送了5美刀，但是相信当时去折腾的人还是很快就会折腾完这5美刀。如果只是为了学习，那么肆无忌惮地调用OpenAI的api的话，还是需要付出很多钱的。现在有了ollama，你可以不需要有昂贵的硬件，不需要懂python，也不需要花一分钱去买api服务，就可以本地使用大模型去进行学习，研究和使用。

ollama的安装，按着[官网](https://ollama.com/download)的实例一步步进行即可。ollama的很多操作非常像docker，如果你平时docker用得比较多，应该会比较亲切。ollama其实使用了llama.cpp作为引擎，就是我们上面提到过的那个llama.cpp提供的main。然后它又模仿了dockerfile，做了一个modelfile的东西，让你通过modelfile可以描述如何将一个语言模型修改为另外一个模型。

ollma的一些常用操作：

In [2]:
# 查看当前本地有哪些模型
!ollama list

# 拉取一个模型
!ollama pull llama3

# 删除一个模型
!ollama rm llama2-chinese:13b

# 启动一个模型
!ollama run llama3 (这里可以直接跟上问题，也可以不跟，进入交互式对话)

# 查看资源占用（最新版本）
!ollama ps

NAME                 	ID          	SIZE  	MODIFIED     
qwen:14b             	80362ced6553	8.2 GB	46 hours ago	
llama3:latest        	a6990ed6be41	4.7 GB	5 days ago  	
gemma:7b             	430ed3535049	5.2 GB	2 months ago	
codellama:13b        	9f438cb9cd58	7.4 GB	3 months ago	
codellama:7b         	8fdf8f752f6e	3.8 GB	3 months ago	
llama2-chinese:13b   	990f930d55c5	7.4 GB	3 months ago	
llama2-chinese:latest	cee11d703eee	3.8 GB	3 months ago	
pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ p

也可以通过restful接口去访问ollama，因为ollama是golang写了，里面自带了一个gin写的服务：

In [8]:
# 生成一个回答
!curl http://localhost:11434/api/generate -d '{"model": "llama3","prompt":"介绍下你自己","stream":false}'

{"model":"llama3","created_at":"2024-05-21T15:56:15.851318233Z","response":"Nice to meet you! 😊 I'm LLaMA, a large language model trained by a team of researcher at Meta AI. My name is derived from \"Large Language Model Application,\" and I'm here to assist and chat with users like you.\n\nI was trained on a massive dataset of text from the internet, which allows me to understand and generate human-like language. My training data includes a wide range of topics, styles, and genres, making me a versatile conversational AI.\n\nHere are some fun facts about me:\n\n1. **I'm a big brain**: I have over 17 billion parameters, which is a lot! This allows me to understand complex sentences, recognize patterns, and generate responses that are both informative and engaging.\n2. **I can converse in multiple styles**: Whether you want a formal conversation or a more casual chat, I can adapt my tone and language to match your preferences.\n3. **I'm a knowledge hub**: I've been trained on a vast amo

In [15]:
# 生成一个对话
!curl http://localhost:11434/api/chat -d '{ \
  "model": "llama3", \
  "messages": [ \
    { "role": "user", "content": "nginx如何优雅停机" } \
  ], \
  "stream":false \
}'

{"model":"llama3","created_at":"2024-05-21T15:59:04.496175848Z","message":{"role":"assistant","content":"😊\n\nTo gracefully shutdown or restart Nginx, you can use the following methods:\n\n**Method 1: Using the `nginx -s` command**\n\nYou can use the `-s` flag followed by one of the following options to control how Nginx shuts down:\n\n* `quit`: Quit immediately\n* `stop`: Stop accepting new connections and finish processing existing requests\n* `restart`: Restart Nginx with a full configuration reload\n\nExample:\n```bash\nnginx -s quit\n```\nThis will shut down Nginx gracefully, allowing existing requests to complete.\n\n**Method 2: Using the `kill` command**\n\nYou can send a signal to the Nginx process using the `kill` command. This method is useful when you want to restart Nginx quickly without waiting for pending requests to finish.\n\nExample:\n```bash\nkill -SIGUSR1 \u003cnginx_pid\u003e\n```\nReplace `\u003cnginx_pid\u003e` with the actual process ID of your Nginx instance.\n\

In [16]:
# embedding
!curl http://localhost:11434/api/embeddings -d '{ \
  "model": "all-minilm", \
  "prompt": "nginx如何优雅停机" \
}'

{"embedding":[-0.09809943288564682,0.2309582680463791,0.23924008011817932,-0.040105562657117844,-0.1428043395280838,-0.056375935673713684,0.6265559792518616,-0.06985802203416824,-0.07132928818464279,-0.3343028426170349,0.07455319166183472,-0.4658896327018738,0.4492717981338501,0.10958538204431534,-0.37046751379966736,-0.20859773457050323,-0.05134684965014458,0.24048760533332825,-0.16589277982711792,0.22192054986953735,-0.1815894991159439,-0.1399330049753189,0.03935636952519417,-0.018299296498298645,-0.013934653252363205,0.16874642670154572,-0.3250887989997864,0.061025336384773254,0.3044147491455078,-0.059163372963666916,-0.16490723192691803,0.4930843412876129,-0.04848567768931389,-0.3056707978248596,0.13449542224407196,-0.08086539804935455,0.256054550409317,-0.12067772448062897,0.12770183384418488,-0.009993761777877808,0.015529081225395203,-0.31010493636131287,0.28019145131111145,-0.07297089695930481,-0.17617563903331757,-0.1567864567041397,0.10576505213975906,-0.15817974507808685,0.27

除了以上三个最常用的api，还有一系列关于镜像拉取，查看，删除等的接口，这些接口在你编写一个需要控制用户镜像的应用的时候还是会有很大用处的，参考[完整的document](https://github.com/ollama/ollama?tab=readme-ov-file)

当然其实ollama官方提供了python和node的包，可以直接使用，但是个人感觉ollama的这两个包有点鸡肋。主要以为langchain的存在，如果你用的是python和nodejs，这两个langchain都有官方支持，不如直接用langchain。如果你用的其他语言，那么ollama也不支持，那可能还是需要自己依靠restful接口去封装。所以ollama的node包和python包我就不介绍了，我们接下来说下ollama的另外一个特色，就是模仿dockerfile的modelfile

相比于dockerfile非常繁多和复杂的指令，ollama的modelfile目前只有7个指令：
##### From
和dockerfile一样，From指定一个基础镜像，这个镜像除了可以是镜像，还可以是gguf文件。

```dockerfile
FROM llama3:8b
```

```dockerfile
FROM ./qwen-frontend.gguf
```

##### PARAMETER
提供了很多可以调整的[底层参数](https://github.com/ollama/ollama/blob/main/docs/modelfile.md)一般来说比较少调整

##### TEMPLATE

TEMPLATE可以用来构造一个提示词模板，也比较少用，因为langchain有更完整的提示词模板

##### SYSTEM
上下文描述，如果你在连续对话的时候需要模型都去记住某个上下文，就可以使用它。比如从上面例子可以看出来llama3经常容你你问中文，它回答英文，这时候可以用SYSTEM指令，用llama3作为基底镜像构造个新的镜像，它英文回答的问题就会立刻缓解很多

```dockerfile
FROM llama3:8b
SYSTEM """你将扮演一个只会说中文的角色，无论我用英文还是中文问你，你都用中文回答我"""
```

##### ADAPTER
这个可以将微调的LoRA适配器文件直接加载到FROM的基础文件上，但是必须是GGML格式，有兴趣可以自己去了解，个人平时没怎么用到过

##### LICENSE
就是LICENSE描述

##### MESSAGE
这个可以提供一段对话的历史，然后将它固化下来。比如我有一段很短的领域知识，LLM根本不可能知道，然后我可以通过对话告诉他，然后再问问题，如果这样的操作非常多，我们可以把提供问题这部分给固化下来。

```dockerfile
FROM llama3
SYSTEM """你将扮演一个只会说中文的角色，无论我用英文还是中文问你，你都用中文回答我"""
MESSAGE user 我将提供一段文字给你，这段文字包含了我同事的一些信息，你根据我提供给你的文字回答问题。
MESSAGE assistant 好的
MESSAGE user 我的室经理是张文涛，我的组长是孙翊浩
```
将上面这段做成镜像后，每次启动都会自动进行前面的对话。然后你就可以直接对提供给他的知识进行问询，而不用每次都去提供一次。

## langchain和RAG
最后来说说langchain和RAG吧。自从昨年ChatGPT出来后不久，就有了langchain。那时候大家对于如何开发一个AI应用还是比较迷茫，langchain抽象出来了AI应用开发的六大组件：Models,Prompts,Indexs,Memory,Chains,Agents。其实严格来说langchain是社区各种想法的一个合集，不使用langchain你也可以进行开发，但是langchain提供了很多便利给你。而且随着社区对于AI应用开发的探索已经越来越多新的架构被提出，langchain也一直在改变，所以这六大组件只是目前langchain的六大组件，将来可能会变成七大，八大，谁知道呢。自从langchain出来之后就一直受到大家的关注和喜爱，github的star数也是一路飙升：
<center>
    <img src="https://llmshare.zhoudailin.cn/pictures/star-history-langchain.png?imageMogr2/thumbnail/800x"/>
</center>

对于langchain开发的具体实施过程，这里就不再赘述，有兴趣可以去自己学习。我们来简单说下什么是RAG（Retrieval Augmented Generation）。同样的我们举个例子：

假设你去问任何一个语言模型，一些你们公司内部的问题，他们都无法回答，因为他们没有学习过相关数据。但是假设，你提供相关数据给他们，就像上面ollama MESSAGE那样，他们就能根据你给的资料去回答问题。那么想想一下，你要做一个知识库，比如我存了一堆PDF电子书，他们可能是某个领域的，比如前端的。我可能把这些书看完了，也可能一个字都没看。我怎么能够让一个语言模型根据这些电子书的内容来回答我的前端问题呢。这里和上面的不一样，因为你不可能把所有的书全部扔个一个大模型，它们往往没有那么大的上下文窗口(现在也有窗口极大的模型，可以装下无限的资料，但是性能消耗依然是个问题)，那么就需要它自己去这些电子书里面找到和你问题最相关的那些片段，然后根据这些片段回答问题。比如我问它前端的Symbol怎么用，它可能就会去我提前存放的电子书里面找和symbol相关的内容，然后根据找到的内容来回答我的问题。这就是所谓检索增强生成(RAG)的含义。

那么如何让LLM自己找到和问题相关的部分呢，这其实就是RAG的核心，也是影响RAG效果的最主要的一步。其实要做到这一点，我们常规也有手段可以做到，比如elasticsearch的倒排索引可以根据问题的关键词去检索片段，但是更多的我们使用embbeding的技术，embedding之前分享已经讲过，这里就只是再简单说下：简单来说就是可以通过embedding将文本片段，视频或者音频啥的，都转化为一个向量，然后你询问问题之后，将问题也转化为一个向量。然后计算各个文字片段向量和问题向量的接近值，就可以知道哪些片段和问题相关。所以embedding算法的选择对于RAG至关重要，当然RAG还有很多优化的手法，比如ReRank，结构化检索库等，以后可以单独分享。

对于RAG还有一个问题，就是我们的向量存在哪里。如果你的被检索数据不多，计算出来的向量当然可以存在内存中。但是往往被检索数据是很多的，这时候一般会使用向量数据库。顾名思义向量数据库就是专门为向量检索而生的，它可以快速进行向量相似度计算，内置了各种向量相似度，是RAG应用除了embedding之外的另外一个核心组件。

最后langchain官方提供了node和python两个语言的，社区提供了各种语言的版本。如果可以，建议直接使用python版本的，因为在AI这一块，python是有最好的生态的。比如举个例子，如果做一个PDF的知识库，你需要将PDF数据读取出来。PDF数据呢可能因你PDF质量，会有各种不同的结果。比如你的PDF做了ocr识别，那么就能读出里面文字信息。比如里面图片很多文字很少，那么你就需要单独思考这些图片怎么处理。同样是官方提供的langchain，在PDF的loader这一块，nodejs就非常简单，只有那么一两个很基本的pdf检索库，出来的效果呢就完全和你PDF本身质量挂钩。但是python版本的pdfloader因为有python生态支持，里面很多深度学习相关的方法，可以帮你做OCR识别，可以是别图片，代码等等等等。同样是PDFLoader，用python版本出来的数据质量会明显更好。同样的，前段时间看java的时候，在spring官网也看到了spring也做了一个叫spring AI的框架，组件基本上都和langchain一样分为那么几类，依托于java的生态，可能在部署或者其他方面会有一定优势，但是在深度学习这一块，个人觉得短时间内其他语言是动摇不了python的地位的，所以如果可以优先使用python版本的。
## 结尾
其实个人感觉折腾了快一年，如果你要做AI相关的开发，不管是学习，还是想给公司落地，或者参与开源生态提升自我，都要想清楚自己是要侧重啥。和四五年前不同，那时候做深度学习，你可能需要懂很多数学，python，机器学习的知识才能摸到门槛。但是现在，即使你完全不会这些，通过社区繁荣的生态，你也可以做点有意思的东西出来，比如我最早看到的bilibili的视频总结，就是一个前端大佬做出来的，用的纯js版本langchain，配合ffmpeg和其他开源社区的项目。还有各个公司，比如之前字节跳动分享的让LLM加入代码检视，这些都挺有意思的，也没有往日那么高的门槛，虽然数学，python，这些知识你知道了对你做大模型应用开发肯定是有益处的，所以如果你有想法，只是害怕或者觉得我办不到，那你大可以去放心尝试。但是作为结束，还是要泼一瓢冷水，也是最近社区看到的一句话：“不要高估一个技术在两年内能带给你的，也不要低估一个技术在十年内能改变的”。